In [1]:
import os
import time

import cv2
import yt_dlp

In [2]:
def process_video_parallel(url, skip_frames, process_number):
    print("in this function")
    cap = cv2.VideoCapture(url)
    num_processes = os.cpu_count()
    frames_per_process = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) // num_processes
    cap.set(cv2.CAP_PROP_POS_FRAMES, frames_per_process * process_number)
    x = 0
    count = 0
    while x < 10 and count < frames_per_process:
        ret, frame = cap.read()
        print("read it")
        if not ret:
            break
        cv2.imshow('video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        filename =r'c:\Users\Nineveh.OConnell\OneDrive - DOT OST\Documents\code\Border-Bottleneck-Management\support_scripts\captured_pngs\shot'+str(x)+'.png'
        x += 1
        cv2.imwrite(filename.format(count), frame)
        count += skip_frames  # Skip 300 frames i.e. 10 seconds for 30 fps
        cap.set(1, count)
    cap.release()

In [3]:
video_url = "https://www.youtube.com/watch?v=0Pg3S6s76IE"  # The Youtube URL


In [4]:
def get_stream_url(youtube_url):
    ydl_opts = {
        'quiet': True,
        'skip_download': True,
        'no_warnings': True,
        'force_generic_extractor': False,
        'format': 'best[ext=mp4][protocol^=http]/best'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        formats = info.get('formats', [info])
        # prefer video-only best mp4 OR best overall
        for f in reversed(formats):
            if f.get('vcodec', '') != 'none' and f.get('acodec', '') == 'none' and 'url' in f:
                return f['url']
        # fallback: best available format
        return formats[-1]['url']

In [ ]:

def capture_frames(stream_url, output_dir='frames', interval=5):
    cap = cv2.VideoCapture(stream_url)
    i = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print('Frame not available, waiting...')
            time.sleep(1)
            continue
        # Save as PNG
        filename =r'c:\Users\Nineveh.OConnell\OneDrive - DOT OST\Documents\code\Border-Bottleneck-Management\support_scripts\captured_pngs'
        cv2.imwrite(f"{filename}/frame_{i:05d}.png", frame)
        print(f"Saved frame {i}")
        time.sleep(interval)
        i += 1

if __name__ == "__main__":
    url = 'https://www.youtube.com/watch?v=0Pg3S6s76IE'  # Replace with your live URL
    stream_url = get_stream_url(url)
    print("Stream URL:", stream_url)
    capture_frames(stream_url)

Stream URL: https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1747781023/ei/P7EsaNSnIdinlu8P0batoAc/ip/152.122.255.5/id/0Pg3S6s76IE.2/itag/270/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hls_chunk_host/rr1---sn-vgqsknsk.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/playlist_duration/30/manifest_duration/30/bui/AecWEAbOlFCeVGpUtz0WvrqbMFM0gnmSnEOhbRkTMQiL0EJBxjM_YaXFrFIGo-4yaxM79i1aZHgZuuus/spc/wk1kZoaLD20Otz60aVsDNnP0h2p6kIrZRTQ57qItA3NESQNF9ec/vprv/1/playlist_type/DVR/initcwndbps/5237500/met/1747759424,/mh/sg/mm/44/mn/sn-vgqsknsk/ms/lva/mv/m/mvi/1/pl/16/rms/lva,lva/dover/13/pacing/0/short_key/1/keepalive/yes/fexp/51466698/mt/1747759011/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgovp,rqh,xpc,playlist_duration,manifest_duration,bui,spc,vprv,playlist_type/sig/AJfQdSswRQIgamXS6CHPmD6EG--Q3VJZRxBe9pibkcWpEZo_JTmbKNgCIQDnVyJYT7hECodJWT-osWZ2XuKXuQa_NQoGUPGe9KqHMg%3D%3D/lsparams/hls_chunk_host,initcwndbps,met,mh